## mle for Bernoulli

- maximum likelihood == minimize Cross Entropy Loss
    - 最大似然估计，估计的是参数（分布的参数），以伯努利分布为例
    - $\{x_i\}_{1,\cdots,N}\sim B(p)$（采样或者观测）
        - $p$ 的概率为1，$1-p$的概率为0，$x_i\in\{0,1\}$
        - 此时需要基于 $\{x_i\}_{1,\cdots,N}$ 来估计 $p$
    - $\ell(p)=\Pi_{i=1}^Np^{x_i}(1-p)^{1-x_i}$（joint probability）

$$
\begin{split}
\log\ell(p)&=\sum_{i=1}^N\log(p^{x_i}(1-p)^{1-x_i})\\
&=\sum_{i=1}^Nx_i\log(p)+(1-x_i)\log(1-p)
\end{split}
$$


- 导数，求极值
    - $\frac{\partial \log\ell(p)}{\partial p}=\frac{\sum_ix_i}{p}-\frac{\sum_i{(1-x_i)}}{1-p}\overset{\text{set}}{=}0$
    - $p=\frac{\sum_ix_i}{N}$（样本均值，sample mean）
    - 极大值还是极小值，可以求二阶导：
        - $\dfrac{\partial^2 \ell(p)}{\partial p^2} = \dfrac{-\sum_{i=1}^n x_i}{p^2} - \dfrac{\sum_{i=1}^n (1-x_i)}{(1-p)^2}$
        - 二阶导小于0，为极大值（参考 $-x^2$）；

## pytorch 梯度下降

### sample from Bernoulli distribution

In [41]:
from scipy import stats
import numpy as np
import torch

In [42]:
p = 0.43
dist = stats.bernoulli(p)
xs = dist.rvs(3000)

In [43]:
xs

array([0, 1, 1, ..., 0, 1, 0])

In [44]:
# sample mean
np.mean(xs)

0.42733333333333334

### mle by pytorch gradient descent

In [45]:
xs_tensor = torch.from_numpy(xs).type(torch.FloatTensor)

In [46]:
xs_tensor

tensor([0., 1., 1.,  ..., 0., 1., 0.])

In [47]:
p_est = torch.rand(1, requires_grad=True)

In [48]:
p_est

tensor([0.2782], requires_grad=True)

In [49]:
lr = 2e-5

$$
\begin{split}
\log\ell(p)&=\sum_{i=1}^N\log(p^{x_i}(1-p)^{1-x_i})\\
&=\sum_{i=1}^Nx_i\log(p)+(1-x_i)\log(1-p)
\end{split}
$$

$$
\frac{\partial \log\ell(p)}{\partial p}=\frac{\sum_ix_i}{p}-\frac{\sum_i{(1-x_i)}}{1-p}
$$

In [52]:
p_est = torch.rand(1, requires_grad=True)
print(p_est)
lr = 2e-5
for epoch in range(100):
    # NLL: negative log likelihood
    # minimize NLL
    NLL = -torch.sum(xs_tensor * torch.log(p_est) + (1-xs_tensor)*torch.log(1-p_est))
    NLL.backward()
    
    if epoch % 5 == 0:
        print(f'p_est:{p_est.data.numpy()}, NLL:{NLL.data.numpy()}, dL/dp: {p_est.grad.data.numpy()}')
        print('\t', torch.sum(xs_tensor)/p_est.data.numpy() - torch.sum(1-xs_tensor)/(1-p_est.data.numpy()))
    
    p_est.data = p_est.data - lr*p_est.grad.data
    p_est.grad.data.zero_()

tensor([0.5192], requires_grad=True)
p_est:[0.5192194], NLL:2098.42724609375, dL/dp: [1104.2644]
	 tensor([-1104.2644])
p_est:[0.4505255], NLL:2050.916748046875, dL/dp: [281.05786]
	 tensor([-281.0579])
p_est:[0.4330856], NLL:2047.8486328125, dL/dp: [70.28589]
	 tensor([-70.2859])
p_est:[0.4287475], NLL:2047.658935546875, dL/dp: [17.321777]
	 tensor([-17.3218])
p_est:[0.42768013], NLL:2047.6468505859375, dL/dp: [4.2504883]
	 tensor([-4.2505])
p_est:[0.42741832], NLL:2047.646728515625, dL/dp: [1.0419922]
	 tensor([-1.0420])
p_est:[0.42735416], NLL:2047.646240234375, dL/dp: [0.2553711]
	 tensor([-0.2554])
p_est:[0.42733842], NLL:2047.646484375, dL/dp: [0.06225586]
	 tensor([-0.0623])
p_est:[0.42733458], NLL:2047.6463623046875, dL/dp: [0.01513672]
	 tensor([-0.0151])
p_est:[0.42733365], NLL:2047.646484375, dL/dp: [0.00390625]
	 tensor([-0.0039])
p_est:[0.4273334], NLL:2047.6463623046875, dL/dp: [0.00097656]
	 tensor([-0.0010])
p_est:[0.42733338], NLL:2047.646240234375, dL/dp: [0.00024414]